In [58]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install captum
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=86a739213436954ed982261bf6c2ac6fa26e893f5add6734f5c7eb1fcf50bb40
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=9ed6bad85760a2e6a3c7e88e742b6069ff1a1fdd95991d23a3d359b60daa4193
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from captum.attr import visualization as viz
from fvcore.nn import FlopCountAnalysis, flop_count_table
from captum.attr import IntegratedGradients, LayerConductance, DeepLift, LayerDeepLift,LayerIntegratedGradients
import itertools
import pandas as pd
import os
import random
import numpy as np
from itertools import product
from torch.utils.data import SequentialSampler, RandomSampler
from torch.utils.data import DataLoader

In [17]:
method_names = ["LayerIntegratedGradients", "LayerDeepLift"]
INPUT_SHAPE= (1, 3, 32, 32)
Training_Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Feature_Attribution_Device = torch.device("cpu")
NUM_EPOCHS=3
NUM_CLASSES=10
IN_CHANNELS=3

# Model structure defination

In [18]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, n1x1, n3x3red, n3x3, n5x5red, n5x5, pool_planes):
        super(InceptionBlock, self).__init__()

        # 1x1 conv branch
        self.conv1x1 = nn.Sequential(
            nn.Conv2d(in_channels, n1x1, kernel_size=1),
            nn.BatchNorm2d(n1x1),
            nn.ReLU(True),
        )

        # 1x1 then 3x3 conv branch
        self.conv1x1_3x3 = nn.Sequential(
            nn.Conv2d(in_channels, n3x3red, kernel_size=1),
            nn.BatchNorm2d(n3x3red),
            nn.ReLU(True),
            nn.Conv2d(n3x3red, n3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(n3x3),
            nn.ReLU(True),
        )

        # 1x1 then 5x5 conv branch
        self.conv1x1_5x5 = nn.Sequential(
            nn.Conv2d(in_channels, n5x5red, kernel_size=1),
            nn.BatchNorm2d(n5x5red),
            nn.ReLU(True),
            nn.Conv2d(n5x5red, n5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(n5x5),
            nn.ReLU(True),
        )

        # 3x3 pool then 1x1 conv branch
        self.pool3x3_conv1x1 = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_planes, kernel_size=1),
            nn.BatchNorm2d(pool_planes),
            nn.ReLU(True),
        )

    def forward(self, x):
        y1 = self.conv1x1(x)
        y2 = self.conv1x1_3x3(x)
        y3 = self.conv1x1_5x5(x)
        y4 = self.pool3x3_conv1x1(x)
        return torch.cat([y1, y2, y3, y4], 1)  # Concatenate on the channel dimension

In [19]:
class Inception(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Inception, self).__init__()

        # Initial convolutional layers
        self.initial_conv_layers = nn.Sequential(
            nn.Conv2d(in_channels, 192, kernel_size=3, padding=1),
            nn.BatchNorm2d(192),
            nn.ReLU(True),
        )

        # Inception blocks in stage 3
        self.inception_block_3a = InceptionBlock(192,  64,  96, 128, 16, 32, 32)
        self.inception_block_3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)

        # Pooling layer between stages
        self.inter_stage_pooling = nn.MaxPool2d(3, stride=2, padding=1)

        # Inception blocks in stage 4
        self.inception_block_4a = InceptionBlock(480, 192,  96, 208, 16,  48,  64)
        self.inception_block_4b = InceptionBlock(512, 160, 112, 224, 24,  64,  64)
        self.inception_block_4c = InceptionBlock(512, 128, 128, 256, 24,  64,  64)
        self.inception_block_4d = InceptionBlock(512, 112, 144, 288, 32,  64,  64)
        self.inception_block_4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)

        # Global average pooling and dropout
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout_layer = nn.Dropout(0.2)

        # Fully connected layer
        self.final_fc_layer = nn.Linear(832, num_classes)

    def forward(self, x):
        x = self.initial_conv_layers(x)

        x = self.inception_block_3a(x)
        x = self.inception_block_3b(x)

        x = self.inter_stage_pooling(x)

        x = self.inception_block_4a(x)
        x = self.inception_block_4b(x)
        x = self.inception_block_4c(x)
        x = self.inception_block_4d(x)
        x = self.inception_block_4e(x)

        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout_layer(x)

        x = self.final_fc_layer(x)
        return x


# Get dataset

In [20]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10('.', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10('.', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


# FLOP Count

In [21]:
def count_flops(model, input_size):
    inputs = torch.randn(input_size)
    flops = FlopCountAnalysis(model, inputs)
    return flop_count_table(flops)

inception_model = Inception(1, 10)
# Assuming the input size for CIFAR-10 (batch size, channels, height, width)
input_size = (1, 1, 28, 28)
flops_table = count_flops(inception_model, input_size)
print(flops_table)

| module                                  | #parameters or shape   | #flops     |
|:----------------------------------------|:-----------------------|:-----------|
| model                                   | 3.382M                 | 0.992G     |
|  initial_conv_layers                    |  2.304K                |  2.107M    |
|   initial_conv_layers.0                 |   1.92K                |   1.355M   |
|    initial_conv_layers.0.weight         |    (192, 1, 3, 3)      |            |
|    initial_conv_layers.0.bias           |    (192,)              |            |
|   initial_conv_layers.1                 |   0.384K               |   0.753M   |
|    initial_conv_layers.1.weight         |    (192,)              |            |
|    initial_conv_layers.1.bias           |    (192,)              |            |
|  inception_block_3a                     |  0.164M                |  0.129G    |
|   inception_block_3a.conv1x1            |   12.48K               |   9.885M   |
|    inception_b

# Train and attribution functions

train and eval function

In [22]:
def train(epoch, model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Train Accuracy = {train_accuracy:.2f}%")

    return train_accuracy

def test(epoch, model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {test_accuracy:.2f}%")

    return test_accuracy


functions for calculate attribution

In [23]:
def print_ig(test_loader, model, device):
    # Move the model to the device (CPU or CUDA)
    model.to(device)

    # Ensure the model is in evaluation mode
    model.eval()

    # Get a single batch from the test loader
    inputs, target_class = next(iter(test_loader))
    inputs = inputs.to(device)

    ig_attributions = {}

    # Iterate through each named module and compute attributions for Conv2d layers with learnable parameters
    for layer_name, layer_module in model.named_modules():
        # Check if the layer is a Conv2d layer with learnable parameters
        if isinstance(layer_module, nn.Conv2d) and any(p.requires_grad for p in layer_module.parameters(recurse=False)):
            # Initialize LayerIntegratedGradients for the layer
            lig = LayerIntegratedGradients(model, layer_module)

            # Compute the attributions for the current layer
            try:
                attributions = lig.attribute(inputs, target=target_class.to(device))
            except Exception as e:
                print(f"Error computing attributions for layer {layer_name}: {e}")
                continue

            # Print out the attributions for the current layer
            print(f'Layer: {layer_name}')
            print(f'Attribution: {attributions.cpu().detach().numpy().sum()}')

            # Store the sum of attributions in the dictionary
            ig_attributions[layer_name] = attributions.cpu().detach().numpy().sum()

            # Free up memory
            del attributions, lig

    return ig_attributions

# Usage example:
# ig_attributions = print_ig(test_loader, model, device)


In [24]:
def print_deeplift(test_loader, model, device):
    # Move the model to the specified device and set it to evaluation mode
    model.to(device).eval()

    # Get a batch of data from the loader
    inputs, target_class = next(iter(test_loader))
    inputs, target_class = inputs.to(device), target_class.to(device)

    dl_attributions = {}

    # Now compute the attributions for Conv2d layers
    for layer_name, layer_module in model.named_modules():
        # Skip the whole model's container and focus on Conv2d layers with learnable parameters
        if isinstance(layer_module, nn.Conv2d) and any(p.requires_grad for p in layer_module.parameters(recurse=False)):
            # Initialize LayerDeepLift with the current layer
            ldl = LayerDeepLift(model, layer_module)

            # Compute the attributions for the current layer
            '''try:
                attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))
            except Exception as e:
                print(f"Error computing attributions for layer {layer_name}: {e}")
                continue'''
            attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))


            # Print out the attributions for the current layer
            print(f'Layer: {layer_name}')
            print(attributions_ldl.cpu().data.numpy().sum())

            dl_attributions[layer_name] = attributions_ldl.cpu().data.numpy().sum()

            del attributions_ldl, ldl

    return dl_attributions

# Usage example:
# dl_attributions = print_deeplift(test_loader, model, device)


# Possible Hyperparameter grid search creation

in hyperparams_list_dict, each hyperparameter has corresponding possible choices as a list, during experiment, given hyperparams sequence, location each hyperperameter's location using hyperparameter encoding function to convert strings or classes back into their index in hyperparams_list_dict

In [32]:
'''sample_hyperparams_list_dict = {
    'initial_lr': [],
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64
}'''

"sample_hyperparams_list_dict = {\n    'initial_lr': [],\n    'optimizer': torch.optim.Adam,  # Example optimizer\n    'criterion': torch.nn.CrossEntropyLoss(),\n    'train_data_used': 0.8,\n    'train_set_shuffle': True,\n    'train_batch_size': 64\n}"

In [33]:
def generate_hyperparameter_combinations(hyperparams):
    """
    Generate a sequence of hyperparameter combinations.

    :param hyperparams: A dictionary where keys are the names of hyperparameters,
                        and values are lists of possible choices for each hyperparameter.
    :return: A list of dictionaries, each representing a unique combination of hyperparameters.
    """
    # Extract the hyperparameter names and their corresponding choices
    keys, values = zip(*hyperparams.items())

    # Generate all possible combinations of hyperparameter values
    all_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    return all_combinations

# Automated Experiments

In [52]:
def get_data_loader(hyperparams, train_dataset, test_dataset):
  shuffle = hyperparams['train_set_shuffle']
  train_batch_size = hyperparams["train_batch_size"]
  train_data_used_num = int(len(train_dataset) * hyperparams["train_data_used"])
  train_indices = torch.randperm(len(train_dataset))[:train_data_used_num]

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=shuffle)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
  return train_loader, test_loader

In [53]:
def run_experiments(hyperparams, train_loader, test_loader, num_epochs=NUM_EPOCHS, num_classes=NUM_CLASSES, in_channels=IN_CHANNELS,):
    # Initialize model
    model = Inception(in_channels, num_classes).to(Training_Device)  # Assuming Inception is defined elsewhere

    # Use the optimizer from hyperparameters
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['initial_lr'])
    criterion = hyperparams['criterion']

    train_accuracy = {}
    test_accuracy = {}

    for epoch in range(num_epochs):
            # Assuming train and test functions are defined elsewhere
            train_acc = train(epoch, model, train_loader, optimizer, criterion, Training_Device)
            test_acc = test(epoch, model, test_loader, criterion, Training_Device)

            train_accuracy["train accuracy epoch"+str(epoch)] = train_acc
            test_accuracy["test accuracy epoch"+str(epoch)] = test_acc




    dl_attributions = print_deeplift(test_loader, model, Feature_Attribution_Device)
    print("\n")

    ig_attributions = print_ig(test_loader, model, Feature_Attribution_Device)
    print("\n")
    return dl_attributions, ig_attributions, train_accuracy, test_accuracy

# Example Usage
hyperparams = {
    'initial_lr': 0.001,
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64

}

train_loader, test_loader = get_data_loader(hyperparams, train_dataset, test_dataset)
#print(run_experiments(num_epochs=0, hyperparams=hyperparams, train_loader=train_loader, test_loader=test_loader))

# Functions for saving attribution

In [54]:
def run_experiments_and_save(hyperparams_combinations, train_dataset, test_dataset, csv_file, num_epochs=NUM_EPOCHS):
    """
    Run experiments for each combination of hyperparameters, get feature layer attributions for DeepLift and Integrated Gradients,
    save the results to a CSV file, and skip any combinations that have already been run.

    :param hyperparams_combinations: List of dictionaries with hyperparameter combinations.
    :param attribution_function: Function to compute feature layer attribution.
    :param csv_file: Path to the CSV file for saving results.
    """

    # Check if the CSV file exists and load existing data
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
    else:
        existing_data = pd.DataFrame()

    for combo in hyperparams_combinations:
        print(combo)
        for i in range(10):  # For each run index
            for method in ['deeplift', 'integrated_gradients']:  # For each method
                # Prepare data for checking if it's already processed
                combo_check = combo.copy()
                combo_check['method'] = method
                combo_check['run'] = i

                # Check if this specific combination is already processed
                if not existing_data.empty and (existing_data[list(combo_check.keys())] == list(combo_check.values())).all(axis=1).any():
                    continue  # Skip if combination is already processed

                # Set seed for reproducibility
                random.seed(i)
                np.random.seed(i)
                torch.manual_seed(i)
                if torch.cuda.is_available():
                    torch.cuda.manual_seed_all(i)

                train_loader, test_loader = get_data_loader(combo, train_dataset, test_dataset)

                # Initialize model
                # Compute attributions
                attr = run_experiments(hyperparams=combo, train_loader=train_loader, test_loader=test_loader)

                # Prepare data for saving
                combo_results = combo.copy()
                combo_results.update(attr)
                combo_results['method'] = method
                combo_results['run'] = i

                # Append results to the existing data
                existing_data = existing_data.append(combo_results, ignore_index=True)

    # Save the data to CSV
    existing_data.to_csv(csv_file, index=False)

# Experiments

In [66]:
hyperparams_choice_list = {
    'initial_lr': [0.001, 0.01],
    'optimizer': [torch.optim.Adam, torch.optim.SGD], # Example optimizer
    'criterion': [torch.nn.CrossEntropyLoss()],
    'train_data_used': [0.5, 1],
    'train_set_shuffle': [True, False],
    'train_batch_size': [64, 16]
}

combinations = generate_hyperparameter_combinations(hyperparams_choice_list)
for combo in combinations:
    print(combo)

{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 0.5, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 0.5, 'train_set_shuffle': True, 'train_batch_size': 16}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 0.5, 'train_set_shuffle': False, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 0.5, 'train_set_shuffle': False, 'train_batch_size': 16}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 

In [67]:
file_path=os.path.join('/content/drive/My Drive/InterpretingNN/code/experiment and result stage2/Cifer-10', 'Cifer-10_result.csv')
print(file_path)

/content/drive/My Drive/InterpretingNN/code/experiment and result stage2/Cifer-10/Cifer-10_result.csv


In [ ]:
run_experiments_and_save(combinations, train_dataset, test_dataset, csv_file=file_path)

{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 0.5, 'train_set_shuffle': True, 'train_batch_size': 64}
Epoch 0: Train Loss = 1.1735, Train Accuracy = 57.15%
Epoch 0: Test Loss = 1.0306, Test Accuracy = 62.32%
